In [1]:
import pandas as pd
import json
import ast
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
with open('Cell_Phones_and_Accessories_5.json','r') as f:
    lines = f.readlines()
    #data = json.load(f)

In [4]:
len(lines)

1128437

In [5]:
# lines[0]

In [5]:
#data = pd.DataFrame(columns=['user_id','product_id','rating','timestamp'])
data = []
cols = ['user_id','product_id','rating','timestamp', 'review']
#cols = list(data.columns)
for i in tqdm(lines):
    dict_i = json.loads(i)
    rating = dict_i['overall']
    user_id = dict_i['reviewerID']
    product_id = dict_i['asin']
    timestamp = dict_i['unixReviewTime']
    if 'reviewText' in dict_i:
        review = dict_i['reviewText']
    else:
        review = ""
    ser = pd.Series([user_id, product_id, rating, timestamp, review], index=cols)
    data.append(ser)

100%|███████████████████████████████████████████████████████████| 1128437/1128437 [04:36<00:00, 4086.31it/s]


In [6]:
data_df = pd.DataFrame(data)

In [7]:
data_df.to_csv('accessories.csv',index=False)

In [8]:
def get_train_test_split(user_df):
    if user_df.shape[0] == 1:
        user_df['split'] = 'train'
        return user_df
    
    train_size=0.8
    train_index = int(train_size*user_df.shape[0])
    df_train = user_df[0:train_index]
    df_rem = user_df[train_index:]
    
    df_train['split'] = 'train'
    df_rem['split'] = 'test'
    
    return pd.concat([df_train,df_rem])

In [9]:
new_data = data_df.groupby(['user_id']).progress_apply(get_train_test_split).reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████| 157212/157212 [03:19<00:00, 787.56it/s]


In [ ]:
train = new_data[new_data['split'] == 'train']
train.to_csv('train.csv',index=False)

In [ ]:
test = new_data[new_data['split'] == 'test']
test.to_csv('test.csv',index=False)

In [ ]:
train.shape, test.shape